In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

In [2]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("numpy")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1732412967921_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt1/yarn/usercache/livy/appcache/application_1732412967921_0001/container_1732412967921_0001_01_000001/tmp/spark-32a452f1-2b3f-4298-ae9a-82953e3babf6
    Can't uninstall 'python-dateutil'. No files were found to uninstall.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.15.30 requires python-dateutil<=2.8.2,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Content_Based_Recommendation").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, explode, split, when, mean, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
from pyspark.ml import Pipeline

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sc.install_pypi_package("fsspec")
sc.install_pypi_package("s3fs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1732412967921_0001/container_1732412967921_0001_01_000001/pyspark.zip/pyspark/context.py", line 2614, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [7]:
import pandas as pd
df_pandas = pd.read_csv("s3://proyecto-mineria-de-datos/anime-dataset-2023.csv")
df_anime = spark.createDataFrame(df_pandas)
df_anime.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+------------------------------+-----+--------------------+--------------------+-----+--------+--------------------+-----------+----------------+--------------------+--------------------+----------------+-----------+-------------+--------------------+------+----------+---------+---------+-------+--------------------+
|anime_id|                Name|        English name|                    Other name|Score|              Genres|            Synopsis| Type|Episodes|               Aired|  Premiered|          Status|           Producers|           Licensors|         Studios|     Source|     Duration|              Rating|  Rank|Popularity|Favorites|Scored By|Members|           Image URL|
+--------+--------------------+--------------------+------------------------------+-----+--------------------+--------------------+-----+--------+--------------------+-----------+----------------+--------------------+--------------------+----------------+-----

In [8]:
# Mostrar el esquema del DataFrame
df_anime.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- anime_id: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- English name: string (nullable = true)
 |-- Other name: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Synopsis: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Episodes: string (nullable = true)
 |-- Aired: string (nullable = true)
 |-- Premiered: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Producers: string (nullable = true)
 |-- Licensors: string (nullable = true)
 |-- Studios: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Rank: string (nullable = true)
 |-- Popularity: long (nullable = true)
 |-- Favorites: long (nullable = true)
 |-- Scored By: string (nullable = true)
 |-- Members: long (nullable = true)
 |-- Image URL: string (nullable = true)

In [9]:
# Número de filas y columnas
num_rows = df_anime.count()
num_cols = len(df_anime.columns)
print(f"Número de filas: {num_rows}, Número de columnas: {num_cols}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

N?mero de filas: 24905, N?mero de columnas: 24

In [10]:
df_anime.describe().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+---------------------+--------------------+-----------------+------------------+-------+---------------------+-------+------------------+------------------+-----------+----------------+----------+-------------------+--------------------+------------+--------+------------+-----------------+------------------+------------------+------------------+------------------+--------------------+
|summary|          anime_id|                 Name|        English name|       Other name|             Score| Genres|             Synopsis|   Type|          Episodes|             Aired|  Premiered|          Status| Producers|          Licensors|             Studios|      Source|Duration|      Rating|             Rank|        Popularity|         Favorites|         Scored By|           Members|           Image URL|
+-------+------------------+---------------------+--------------------+-----------------+------------------+-------+---------------------+-------+------------------+---

In [11]:
# Lista de columnas y sus tipos
for col, dtype in df_anime.dtypes:
    print(f"Columna: {col}, Tipo: {dtype}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Columna: anime_id, Tipo: bigint
Columna: Name, Tipo: string
Columna: English name, Tipo: string
Columna: Other name, Tipo: string
Columna: Score, Tipo: string
Columna: Genres, Tipo: string
Columna: Synopsis, Tipo: string
Columna: Type, Tipo: string
Columna: Episodes, Tipo: string
Columna: Aired, Tipo: string
Columna: Premiered, Tipo: string
Columna: Status, Tipo: string
Columna: Producers, Tipo: string
Columna: Licensors, Tipo: string
Columna: Studios, Tipo: string
Columna: Source, Tipo: string
Columna: Duration, Tipo: string
Columna: Rating, Tipo: string
Columna: Rank, Tipo: string
Columna: Popularity, Tipo: bigint
Columna: Favorites, Tipo: bigint
Columna: Scored By, Tipo: string
Columna: Members, Tipo: bigint
Columna: Image URL, Tipo: string

In [12]:
# Agrupar animes por tipo
df_type=df_anime.groupBy("Type").count().orderBy(F.desc("count"))
df_type.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+
|   Type|count|
+-------+-----+
|     TV| 7597|
|  Movie| 4381|
|    OVA| 4076|
|    ONA| 3533|
|  Music| 2686|
|Special| 2558|
|UNKNOWN|   74|
+-------+-----+

In [13]:
from pyspark.sql.functions import col, when, split, explode
# Reemplazar "UNKNOWN" por NULL (vacío) en la columna "Genres"
df_anime = df_anime.withColumn("Genres", when(col("Genres") == "UNKNOWN", None).otherwise(col("Genres")))
# Dividir la columna "Genres" en una lista de géneros
df_genres = df_anime.withColumn("GenreArray", split(col("Genres"), ", "))
# Explode: Expandir los géneros en filas individuales
df_exploded = df_genres.withColumn("Genre", explode(col("GenreArray")))
# Contar las ocurrencias de cada género
genre_counts = df_exploded.groupBy("Genre").count().orderBy(col("count").desc())
# Mostrar el resultado
genre_counts.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----+
|        Genre|count|
+-------------+-----+
|       Comedy| 7142|
|      Fantasy| 5306|
|       Action| 4730|
|    Adventure| 3842|
|       Sci-Fi| 3091|
|        Drama| 2836|
|      Romance| 2063|
|Slice of Life| 1755|
| Supernatural| 1494|
|       Hentai| 1486|
|      Mystery|  847|
|  Avant Garde|  804|
|        Ecchi|  795|
|       Sports|  771|
|       Horror|  534|
|     Suspense|  242|
|Award Winning|  241|
|    Boys Love|  169|
|      Gourmet|  146|
|   Girls Love|  113|
+-------------+-----+
only showing top 20 rows

In [14]:
# Filtrar filas con valor "UNKNOWN"
unknown_score = df_anime.filter(col("Score") == "UNKNOWN").count()
print(f"Número de animes con Score 'UNKNOWN': {unknown_score}")
unknown_episodes = df_anime.filter(col("Episodes") == "UNKNOWN").count()
print(f"Número de animes con Episodes 'UNKNOWN': {unknown_episodes}")
unknown_type = df_anime.filter(col("Type") == "UNKNOWN").count()
print(f"Número de animes con Type 'UNKNOWN': {unknown_type}")
unknown_aired = df_anime.filter(col("Aired") == "UNKNOWN").count()
print(f"Número de animes con Aired 'UNKNOWN': {unknown_aired}")
unknown_status = df_anime.filter(col("Status") == "UNKNOWN").count()
print(f"Número de animes con Status 'UNKNOWN': {unknown_status}")
unknown_studios = df_anime.filter(col("Studios") == "UNKNOWN").count()
print(f"Número de animes con Studios 'UNKNOWN': {unknown_studios}")
unknown_source = df_anime.filter(col("Source") == "UNKNOWN").count()
print(f"Número de animes con Source 'UNKNOWN': {unknown_source}")
unknown_duration = df_anime.filter(col("Duration") == "Unknown").count()
print(f"Número de animes con Duration 'UNKNOWN': {unknown_duration}")
unknown_rating = df_anime.filter(col("Rating") == "UNKNOWN").count()
print(f"Número de animes con Rating 'UNKNOWN': {unknown_rating}")
unknown_pop = df_anime.filter(col("Popularity") == "Unknown").count()
print(f"Número de animes con Popularity 'UNKNOWN': {unknown_pop}")
unknown_mem = df_anime.filter(col("Members") == "UNKNOWN").count()
print(f"Número de animes con Members 'UNKNOWN': {unknown_mem}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

N?mero de animes con Score 'UNKNOWN': 9213
N?mero de animes con Episodes 'UNKNOWN': 611
N?mero de animes con Type 'UNKNOWN': 74
N?mero de animes con Aired 'UNKNOWN': 0
N?mero de animes con Status 'UNKNOWN': 0
N?mero de animes con Studios 'UNKNOWN': 10526
N?mero de animes con Source 'UNKNOWN': 0
N?mero de animes con Duration 'UNKNOWN': 663
N?mero de animes con Rating 'UNKNOWN': 669
N?mero de animes con Popularity 'UNKNOWN': 0
N?mero de animes con Members 'UNKNOWN': 0

In [15]:
# Episodes a tipo float
df_anime = df_anime.withColumn("Episodes", col("Episodes").cast("float"))
# Filtrar las filas que tienen valores numéricos
df_numeric_episodes = df_anime.filter(col("Episodes").isNotNull())
# Calcular la media de los episodios
mean_episodes = df_numeric_episodes.select(mean(col("Episodes"))).collect()[0][0]
# Reemplazar los valores "UNKNOWN" por la media calculada
df_anime = df_anime.withColumn("Episodes", when(col("Episodes").isNull(), mean_episodes).otherwise(col("Episodes")))
# Ordenar por Score en orden descendente, seleccionar columnas y limitar a 15 resultados
top_15_animes = (df_anime.orderBy(col("Episodes").desc()).select("Name", "Episodes").limit(15))
# Mostrar el resultado
top_15_animes.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------+--------+
|Name                                 |Episodes|
+-------------------------------------+--------+
|Lan Mao                              |3057.0  |
|Oyako Club                           |1818.0  |
|Doraemon (1979)                      |1787.0  |
|Daomei Xiansheng                     |1664.0  |
|Kirin Monoshiri Yakata               |1565.0  |
|Manga Nippon Mukashibanashi (1976)   |1471.0  |
|Hoka Hoka Kazoku                     |1428.0  |
|Kirin Ashita no Calendar             |1306.0  |
|Monoshiri Daigaku: Ashita no Calendar|1274.0  |
|Sekai Monoshiri Ryoko                |1006.0  |
|Kaixin Hanzi                         |1000.0  |
|Shuimu Baobao Kan Shijie             |800.0   |
|Kotowaza House                       |773.0   |
|Asa da yo! Kaishain                  |744.0   |
|Shima Shima Tora no Shimajirou       |726.0   |
+-------------------------------------+--------+

In [16]:
# Reemplazar "UNKNOWN" por NULL
df_anime = df_anime.withColumn("Type", when(col("Type") == "UNKNOWN", None).otherwise(col("Type")))
df_anime = df_anime.withColumn("Duration", when(col("Duration") == "Unknown", None).otherwise(col("Duration")))
df_anime = df_anime.withColumn("Rating", when(col("Rating") == "UNKNOWN", None).otherwise(col("Rating")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# DataFrame con las columnas definitivas
data = df_anime.select("anime_id", "Name", "Genres", "Synopsis", "Type", "Episodes", "Aired", 
                       "Status", "Source", "Duration", "Rating", "Popularity", "Members")

for col, dtype in data.dtypes:
    print(f"Columna: {col}, Tipo: {dtype}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Columna: anime_id, Tipo: bigint
Columna: Name, Tipo: string
Columna: Genres, Tipo: string
Columna: Synopsis, Tipo: string
Columna: Type, Tipo: string
Columna: Episodes, Tipo: double
Columna: Aired, Tipo: string
Columna: Status, Tipo: string
Columna: Source, Tipo: string
Columna: Duration, Tipo: string
Columna: Rating, Tipo: string
Columna: Popularity, Tipo: bigint
Columna: Members, Tipo: bigint

In [18]:
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import IntegerType

# Variables tipo numéricas
columns_to_normalize = ["Episodes", "Popularity", "Members"]
for column in columns_to_normalize:
    data = data.withColumn(column, col(column).cast("float"))
# Vectorizar columnas numéricas
assembler = VectorAssembler(inputCols=columns_to_normalize, outputCol="features_vector")
data = assembler.transform(data)

# Función UDF para obtener la longitud de un vector
def get_vector_size(v):
    return len(v)
# Registrar la UDF
get_vector_size_udf = udf(get_vector_size, IntegerType())
# Aplicar la UDF para obtener el tamaño del vector
data = data.withColumn("vector_size", get_vector_size_udf("features_vector"))

# Verificar los resultados
data.select("features_vector", "vector_size").show(truncate=False)

# Eliminar la columna de verificación
data = data.drop("vector_size")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------+-----------+
|features_vector                    |vector_size|
+-----------------------------------+-----------+
|[26.0,43.0,1771505.0]              |3          |
|[1.0,602.0,360978.0]               |3          |
|[26.0,246.0,727252.0]              |3          |
|[26.0,1795.0,111931.0]             |3          |
|[52.0,5126.0,15001.0]              |3          |
|[145.0,1252.0,177688.0]            |3          |
|[24.0,862.0,260166.0]              |3          |
|[52.0,4212.0,24172.0]              |3          |
|[24.0,1273.0,173710.0]             |3          |
|[74.0,142.0,1013100.0]             |3          |
|[220.0,8.0,2717330.0]              |3          |
|[14.913517951965332,20.0,2168904.0]|3          |
|[178.0,1246.0,178273.0]            |3          |
|[12.0,8288.0,4581.0]               |3          |
|[26.0,687.0,320203.0]              |3          |
|[24.0,1546.0,134894.0]             |3          |
|[22.0,935.0,239423.0]              |3          |


In [19]:
# Aplicar MinMaxScaler
scaler = MinMaxScaler(inputCol="features_vector", outputCol="scaled_features")
scaler_model = scaler.fit(data)
data = scaler_model.transform(data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## TF-IDF 

In [20]:
from pyspark.sql.functions import concat_ws

# Concatenar todas las columnas tipo string
data = data.withColumn("combined_text", concat_ws(" ", "Name", "Genres", "Synopsis", "Type", 
                                                  "Aired", "Status", "Source", "Duration", "Rating"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
from pyspark.sql.functions import lower, regexp_replace, split
from pyspark.ml.feature import StopWordsRemover

# Texto en minusculas y eliminar caracteres especiales
data = data.withColumn("processed_text", lower(col("combined_text")))
data = data.withColumn("processed_text", regexp_replace(col("processed_text"), "[^a-zA-Z\\s]", ""))

# Convertir la columna 'processed_text' en un array de palabras
data = data.withColumn("words", split(col("processed_text"), " "))

# Eliminar stopwords 
remover = StopWordsRemover(inputCol="words", outputCol="filtered_text")
data = remover.transform(data)

data.select("processed_text", "words", "filtered_text").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml import Pipeline

# Crear un HashingTF para vectorizar el texto procesado
hashingTF = HashingTF(inputCol="filtered_text", outputCol="raw_features", numFeatures=2000)

# Calcular TF-IDF
idf = IDF(inputCol="raw_features", outputCol="features")

# Pipeline para las transformaciones
pipeline = Pipeline(stages=[hashingTF, idf])

model = pipeline.fit(data)
data = model.transform(data)

data.select("anime_id", "features").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [23]:
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
import numpy as np

# Función UDF para calcular la similitud de coseno
def cosine_similarity(vec1, vec2):
    # Convertir vectores de SparseVector a arrays
    vec1_array = np.array(vec1.toArray())
    vec2_array = np.array(vec2.toArray())
    
    # Calcular el producto punto y las longitudes de los vectores
    dot_product = np.dot(vec1_array, vec2_array)
    norm_a = np.linalg.norm(vec1_array)
    norm_b = np.linalg.norm(vec2_array)
    
    # Calcular la similitud de coseno
    return float(dot_product / (norm_a * norm_b)) if norm_a and norm_b else 0.0

# Registrar la UDF de similitud de coseno
cosine_udf = udf(cosine_similarity, DoubleType())

# matriz de similitudes
cosine_sim_matrix = data.alias("df1").join(data.alias("df2"), col("df1.anime_id") != col("df2.anime_id")) \
    .withColumn("cosine_sim", cosine_udf(col("df1.features"), col("df2.features")))

cosine_sim_matrix.select("df1.anime_id", "df2.anime_id", "cosine_sim").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+--------------------+
|anime_id|anime_id|cosine_sim          |
+--------+--------+--------------------+
|1       |5       |0.27717354606684236 |
|1       |6       |0.0690976769271532  |
|1       |7       |0.09294476362953948 |
|1       |8       |0.006376256346380185|
|1       |15      |0.056060148310866184|
|1       |16      |0.04618445294864644 |
|1       |17      |0.01772991669864651 |
|1       |18      |0.02069310117986595 |
|1       |19      |0.04082176935981903 |
|1       |20      |0.030847117965757585|
|1       |21      |0.04204461941452779 |
|1       |22      |0.011677840543964705|
|1       |23      |0.04325172642824893 |
|1       |24      |0.05588853133369697 |
|1       |25      |0.041362318687139306|
|1       |26      |0.05154599008042234 |
|1       |27      |0.09314454749797456 |
|1       |28      |0.03713050452584757 |
|1       |29      |0.04007128724352351 |
|1       |30      |0.039689196822682314|
+--------+--------+--------------------+
only showing top

In [24]:
from pyspark.sql.functions import col, desc

# Función para obtener recomendaciones basadas en la similitud de coseno
def get_recommendations(anime_name, cosine_sim_matrix, df, top_n=10):
    # Buscar el anime_id basado en el nombre
    anime_id_row = df.filter(col("Name") == anime_name).select("anime_id").first()
    
    if anime_id_row is None:
        return "Anime not found"
    
    anime_id = anime_id_row["anime_id"]
    
    # Filtrar las similitudes para el anime_id dado
    recommendations = cosine_sim_matrix.filter(col("df1.anime_id") == anime_id) \
        .orderBy(desc("cosine_sim")) \
        .select("df2.anime_id", "df2.Name", "cosine_sim")
    
    # Mostrar las top_n recomendaciones
    recommendations = recommendations.limit(top_n)
    
    return recommendations

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Ejemplo:
recommended_animes = get_recommendations(anime_name="Tensei shitara Slime Datta Ken", cosine_sim_matrix=cosine_sim_matrix, df=data, top_n=20)
recommended_animes.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------------------------------------------------------+-------------------+
|anime_id|Name                                                              |cosine_sim         |
+--------+------------------------------------------------------------------+-------------------+
|45753   |Tensei shitara Slime Datta Ken: Kanwa - Hinata Sakaguchi          |0.30129526747204816|
|49318   |Tensei shitara Slime Datta Ken 2nd Season: Kanwa - Veldora Nikki 2|0.26550321253120623|
|37985   |Aggressive Retsuko (ONA) 2nd Season                               |0.26027111840745987|
|39551   |Tensei shitara Slime Datta Ken 2nd Season                         |0.2587535974467901 |
|39607   |Tensei shitara Slime Datta Ken: Kanwa - Veldora Nikki             |0.25143453132829896|
|40215   |Aggressive Retsuko (ONA) 3rd Season                               |0.2495441762293923 |
|46387   |Zhiduo Xingjimu                                                   |0.2457635910277627 |
|53580   |Tensei shi

In [29]:
data_flattened = data.select(
    'anime_id',
    'features'
)

data_flattened = data_flattened.withColumn("features", col("features").cast("string"))
data_flattened.write.csv("s3://proyecto-mineria-de-datos/anime_vector2.csv", header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
model.write().overwrite().save("s3://proyecto-mineria-de-datos/tfidf_pipeline_model")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…